In [55]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [56]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [57]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [58]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values
Discarding below variables duw to large number of categories 
1. start_station_id
2. end_station_id
3. bikeid

In [59]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [60]:
df = tripdata[selected_features]
df.shape

(315433, 22)

In [61]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### AMputation with 10% NA values

In [62]:
## get the sample size such as 1%, 5%, 10%,20% etc
def getSampleSize(df, perc):
    return round(perc/100 * df.shape[0])

def getAnIndex(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def induceMissingValues(df, col_name, perc):
    random.seed(100)
    index = getAnIndex(df.index)
    sample_size = getSampleSize(df, perc)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        df.loc[i, col_name]= np.NaN
    return df

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def induceMissingValuesForDrop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        y_train.loc[i] = np.NaN
        X_train.loc[i,col_name]= np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [63]:
def sample_imputation_X(var, X_train, X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [64]:
list_comp_time = []
def ampute_each_variables(variable, method,X_train,y_train,X_test,y_test):
    if method == "remove":
        X_train , y_train = induceMissingValuesForDrop(X_train,y_train,variable,perc=1)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = induceMissingValuesForDrop(X_test,y_test,variable,perc=1)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train  = induceMissingValues(X_train,variable,perc=1)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test  = induceMissingValues(X_test,variable,perc=1)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    start = datetime.datetime.now()
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
        
    end = datetime.datetime.now()
    comp_time = (end-start)
    print(f"Time taken simulate for variable {var} = ", comp_time)
    list_comp_time.append(comp_time)    
            
    return X_train , y_train, X_test , y_test

In [65]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [66]:
def model_evaluation(X_train,y_train):
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    return model_lr

In [67]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [68]:
model_drop_metrics ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_drop_metrics[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  2523
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  631
X_test Amputed value count of varaible  631
Simulate Imputation for NA
Train after NA value removal Imputation (249823, 21) (249823,)
Test after NA value removal Imputation (62456, 21) (62456,)
Time taken simulate for variable start_lat =  0:00:00.371626
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  2523
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  631
X_test Ampu

X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  631
X_test Amputed value count of varaible  631
Simulate Imputation for NA
Train after NA value removal Imputation (249823, 21) (249823,)
Test after NA value removal Imputation (62456, 21) (62456,)
Time taken simulate for variable solarradiation =  0:00:00.345874
Now running model is .... conditions
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  2523
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  631
X_test Amputed value count of varaible  631
Simulate Imputation for NA
Train after NA value removal Imputation (249823, 21) (249823,)
Test after NA value removal Imputation (62456, 21) (62456,)
Time taken simulate 

In [72]:
print("averge time to run drop is : ", 0.350272, "seconds")
list_comp_time

averge time to run drop is :  0.350272 seconds


[datetime.timedelta(microseconds=371626),
 datetime.timedelta(microseconds=342892),
 datetime.timedelta(microseconds=281910),
 datetime.timedelta(microseconds=453944),
 datetime.timedelta(microseconds=336407),
 datetime.timedelta(microseconds=429527),
 datetime.timedelta(microseconds=387700),
 datetime.timedelta(microseconds=367006),
 datetime.timedelta(microseconds=322998),
 datetime.timedelta(microseconds=380984),
 datetime.timedelta(microseconds=316686),
 datetime.timedelta(microseconds=361587),
 datetime.timedelta(microseconds=320960),
 datetime.timedelta(microseconds=345874),
 datetime.timedelta(microseconds=314889),
 datetime.timedelta(microseconds=291064),
 datetime.timedelta(microseconds=357737),
 datetime.timedelta(microseconds=385374),
 datetime.timedelta(microseconds=292045),
 datetime.timedelta(microseconds=397659),
 datetime.timedelta(microseconds=296844)]

In [70]:
model_drop_metrics

{'start_lat': {'MAE': 1.573023484558106,
  'MSE': 5.00423392065178,
  'RMSE': 2.23701451060376,
  'R2': 0.5266292556125269},
 'start_lon': {'MAE': 1.573023484558106,
  'MSE': 5.00423392065178,
  'RMSE': 2.23701451060376,
  'R2': 0.5266292556125269},
 'end_lat': {'MAE': 1.573023484558106,
  'MSE': 5.00423392065178,
  'RMSE': 2.23701451060376,
  'R2': 0.5266292556125269},
 'end_lon': {'MAE': 1.573023484558106,
  'MSE': 5.00423392065178,
  'RMSE': 2.23701451060376,
  'R2': 0.5266292556125269},
 'usertype': {'MAE': 1.573023484558106,
  'MSE': 5.00423392065178,
  'RMSE': 2.23701451060376,
  'R2': 0.5266292556125269},
 'hour': {'MAE': 1.573023484558106,
  'MSE': 5.00423392065178,
  'RMSE': 2.23701451060376,
  'R2': 0.5266292556125269},
 'temp': {'MAE': 1.573023484558106,
  'MSE': 5.00423392065178,
  'RMSE': 2.23701451060376,
  'R2': 0.5266292556125269},
 'feelslike': {'MAE': 1.573023484558106,
  'MSE': 5.00423392065178,
  'RMSE': 2.23701451060376,
  'R2': 0.5266292556125269},
 'dew': {'MAE':

### Mode Replacement

In [73]:
model_metrics_mode ={}
list_comp_time = []
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mode[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.283157
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lon =  0:00:00.300845
Now model is running for varaible ==== e

Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable holiday =  0:00:00.380171
Now model is running for varaible ==== day
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
Time taken simulate for variable day =  0:00:00.330540
Now model is running for varaible ==== month
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train 

In [76]:
print("averge time to run mode is : ", 0.320927,"seconds")
list_comp_time

averge time to run mode is :  0.320927 seconds


[datetime.timedelta(microseconds=283157),
 datetime.timedelta(microseconds=300845),
 datetime.timedelta(microseconds=293087),
 datetime.timedelta(microseconds=236015),
 datetime.timedelta(microseconds=290301),
 datetime.timedelta(microseconds=289117),
 datetime.timedelta(microseconds=393326),
 datetime.timedelta(microseconds=316816),
 datetime.timedelta(microseconds=390775),
 datetime.timedelta(microseconds=301101),
 datetime.timedelta(microseconds=303471),
 datetime.timedelta(microseconds=312704),
 datetime.timedelta(microseconds=294115),
 datetime.timedelta(microseconds=361837),
 datetime.timedelta(microseconds=293029),
 datetime.timedelta(microseconds=328778),
 datetime.timedelta(microseconds=304194),
 datetime.timedelta(microseconds=380171),
 datetime.timedelta(microseconds=330540),
 datetime.timedelta(microseconds=349467),
 datetime.timedelta(microseconds=386630)]

In [75]:
model_metrics_mode

{'start_lat': {'MAE': 1.5723051733197362,
  'MSE': 4.999626510089749,
  'RMSE': 2.23598446105731,
  'R2': 0.5272344551340956},
 'start_lon': {'MAE': 1.5723482616373556,
  'MSE': 4.9998605199374,
  'RMSE': 2.236036788592129,
  'R2': 0.5272123271225521},
 'end_lat': {'MAE': 1.5723165458096728,
  'MSE': 4.9995519590925355,
  'RMSE': 2.235967790262761,
  'R2': 0.5272415046892467},
 'end_lon': {'MAE': 1.5723300036668895,
  'MSE': 4.999584618650703,
  'RMSE': 2.2359750934772737,
  'R2': 0.527238416395795},
 'usertype': {'MAE': 1.572419862390175,
  'MSE': 4.999829468190998,
  'RMSE': 2.236029845102922,
  'R2': 0.527215263381046},
 'hour': {'MAE': 1.5724188623066166,
  'MSE': 5.000332253826668,
  'RMSE': 2.2361422704798253,
  'R2': 0.5271677198846431},
 'temp': {'MAE': 1.5724215285461476,
  'MSE': 5.000114797855349,
  'RMSE': 2.236093646933274,
  'R2': 0.5271882825587864},
 'feelslike': {'MAE': 1.572435206329298,
  'MSE': 5.000185990010771,
  'RMSE': 2.2361095657437655,
  'R2': 0.5271815506162

### Mean Replacement

In [77]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','dew','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

In [78]:
model_metrics_mean ={}
list_comp_time =[]
for var in variables_for_mean_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_mean[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.354043
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
Time taken simulate for variable start_lon =  0:00:00.303059
Now model is running for varaible ==== e

In [79]:
print("averge time to run mean is : ", 0.306396, "seconds")
list_comp_time

averge time to run mean is :  2.7004244


[datetime.timedelta(microseconds=354043),
 datetime.timedelta(microseconds=303059),
 datetime.timedelta(microseconds=319891),
 datetime.timedelta(microseconds=300673),
 datetime.timedelta(microseconds=367143),
 datetime.timedelta(microseconds=229735),
 datetime.timedelta(microseconds=293002),
 datetime.timedelta(microseconds=273040),
 datetime.timedelta(microseconds=371445),
 datetime.timedelta(microseconds=277158),
 datetime.timedelta(microseconds=252851),
 datetime.timedelta(microseconds=285913),
 datetime.timedelta(microseconds=340775),
 datetime.timedelta(microseconds=265074),
 datetime.timedelta(microseconds=362139)]

In [80]:
model_metrics_mean

{'start_lat': {'MAE': 1.5723028869826516,
  'MSE': 4.999614575672232,
  'RMSE': 2.2359817923391576,
  'R2': 0.5272355836546738},
 'start_lon': {'MAE': 1.5723461241072976,
  'MSE': 4.9998601479731155,
  'RMSE': 2.2360367054172245,
  'R2': 0.5272123622955589},
 'end_lat': {'MAE': 1.5723121180253157,
  'MSE': 4.9995328416028055,
  'RMSE': 2.2359635152664734,
  'R2': 0.527243312442372},
 'end_lon': {'MAE': 1.5723241178832696,
  'MSE': 4.999559508289178,
  'RMSE': 2.2359694783894475,
  'R2': 0.5272407908359105},
 'hour': {'MAE': 1.5723609032674861,
  'MSE': 4.99991571126535,
  'RMSE': 2.236049129886316,
  'R2': 0.5272071082210645},
 'temp': {'MAE': 1.5723752525321657,
  'MSE': 4.999882767175554,
  'RMSE': 2.236041763289665,
  'R2': 0.5272102234198762},
 'feelslike': {'MAE': 1.5724278105372327,
  'MSE': 5.000154546519626,
  'RMSE': 2.2361025348851125,
  'R2': 0.5271845239182386},
 'dew': {'MAE': 1.572315296246298,
  'MSE': 4.999562631568011,
  'RMSE': 2.2359701768064824,
  'R2': 0.5272404954

### Sampling Replacement

In [81]:
model_metrics_sample ={}
list_comp_time =[]
for var in features_for_amputation:
    print("Now model is running for varaible ====", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = model_evaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    model_metrics_sample[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    
    del X_train
    del X_test
    del y_train
    del y_test

Now model is running for varaible ==== start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simple Imputation using sampling
Time taken simulate for variable start_lat =  0:00:00.014990
Now model is running for varaible ==== start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simple Imputation using sampling
Time taken simulate for variable start_lon =  0:00:00.015032
Now model is running for varaible ==== end_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
Test set after Amputation (63087

In [82]:
print("averge time to run sample is : ", 0.35091,"seconds")
list_comp_time

averge time to run sample is :  2.68674819


[datetime.timedelta(microseconds=14990),
 datetime.timedelta(microseconds=15032),
 datetime.timedelta(microseconds=15012),
 datetime.timedelta(microseconds=15600),
 datetime.timedelta(microseconds=62628),
 datetime.timedelta(microseconds=38952),
 datetime.timedelta(microseconds=24941),
 datetime.timedelta(microseconds=15037),
 datetime.timedelta(microseconds=14037),
 datetime.timedelta(microseconds=15000),
 datetime.timedelta(microseconds=16019),
 datetime.timedelta(microseconds=15008),
 datetime.timedelta(microseconds=15000),
 datetime.timedelta(microseconds=18007),
 datetime.timedelta(microseconds=57635),
 datetime.timedelta(microseconds=16951),
 datetime.timedelta(microseconds=52521),
 datetime.timedelta(microseconds=67503),
 datetime.timedelta(microseconds=103569),
 datetime.timedelta(microseconds=79712),
 datetime.timedelta(microseconds=63771)]

In [83]:
model_metrics_sample

{'start_lat': {'MAE': 1.5722983733789146,
  'MSE': 4.999546367423081,
  'RMSE': 2.235966539871087,
  'R2': 0.5272420334384744},
 'start_lon': {'MAE': 1.5723975220621318,
  'MSE': 5.000145791081451,
  'RMSE': 2.2361005771390183,
  'R2': 0.5271853518339821},
 'end_lat': {'MAE': 1.5723151521444365,
  'MSE': 4.9994699357634085,
  'RMSE': 2.2359494483917586,
  'R2': 0.5272492608293919},
 'end_lon': {'MAE': 1.5724606800432965,
  'MSE': 4.999861763928357,
  'RMSE': 2.236037066760826,
  'R2': 0.5272122094905527},
 'usertype': {'MAE': 1.572755840181274,
  'MSE': 5.0019820276404054,
  'RMSE': 2.2365111284409935,
  'R2': 0.5270117169883501},
 'hour': {'MAE': 1.5725154389696456,
  'MSE': 5.000864240041558,
  'RMSE': 2.2362612190979743,
  'R2': 0.5271174151764342},
 'temp': {'MAE': 1.572422111261905,
  'MSE': 5.000000882530477,
  'RMSE': 2.2360681748395947,
  'R2': 0.5271990544115484},
 'feelslike': {'MAE': 1.5724554845240328,
  'MSE': 5.000349793197187,
  'RMSE': 2.2361461922685617,
  'R2': 0.5271